In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

In [65]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
N = train.shape[0]

In [66]:
frac = 0.1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
#train = train.tail(n=int(N * frac))
#train = train.sample(frac=frac, random_state=57)

N = train.shape[0]

train['week'] = train['week'] - train['shift']

for i in range(N):
    if train['week'].values[i] <= 0:
        train['week'].values[i] = (train['week'].values[i] + 53) % 54
        train['year'].values[i] = (train['year'].values[i] - 1)

train = train.drop(['shift'], axis=1)

train = train.sort(['year', 'week'])

X = train.drop(['Num', 'y'], axis=1)

y = train['y'].reshape(N, 1)
y = [elem[0] for elem in y]

/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [67]:
#F = X.drop(['item_id', 'year', 'week'], axis=1)
F = np.array(X)
# weeks = [[] for f in F]
# for fi, f in enumerate(F):
#     for i in range(len(f) - 7):
#         s = 0
#         for x in range(7):
#             s = s + f[i + x]
#         weeks[fi].append(s)

In [94]:
import matplotlib.pyplot as plt

In [101]:
plt.scatter(range(len(y)), y, color='red')
for i in range(len(y)):
    plt.scatter([i] * len(weeks[i]), weeks[i], color='blue', alpha=0.02)
plt.show()

KeyboardInterrupt: 

In [96]:
results = []
for i, ans in enumerate(y):
    for j, w in enumerate(weeks[i]):
        if ans == w:
            #print i, j
            results.append(i)
from collections import Counter
counter = Counter(results)
print len(counter)
print counter

1
Counter({6671: 2})


In [20]:
results = []
for i, ans in enumerate(y):
    for j, f in enumerate(F[i][3:]):
        if ans == f:
            results.append((F[i][0], F[i][1], F[i][2], j))

In [68]:
d = {}
for i, ans in enumerate(y):
    for j, f in enumerate(F[i][3:]):
        if ans == f:
            year = F[i][0]
            week = F[i][1]
            item_id = F[i][2]
            if not((item_id, year) in d.keys()):
                if j > 30:
                    d[(item_id, year)] = (week, j - 30)
                else:
                    d[(item_id, year)] = (week, j)
print d

{(20438756.0, 2013.0): (28.0, 29), (20438809.0, 2013.0): (3.0, 21), (20449962.0, 2014.0): (5.0, 28), (20433851.0, 2014.0): (9.0, 9), (20438923.0, 2014.0): (42.0, 28), (20427503.0, 2013.0): (7.0, 12), (20427504.0, 2013.0): (46.0, 25), (20438690.0, 2014.0): (1.0, 6), (20442315.0, 2012.0): (53.0, 10), (20433852.0, 2013.0): (45.0, 10), (20442153.0, 2012.0): (53.0, 16), (20441455.0, 2012.0): (52.0, 20), (20439113.0, 2013.0): (6.0, 16), (20438810.0, 2014.0): (1.0, 11), (20438691.0, 2014.0): (29.0, 4), (20448086.0, 2014.0): (1.0, 1), (20438924.0, 2013.0): (19.0, 21), (20440441.0, 2014.0): (35.0, 25), (20438731.0, 2013.0): (11.0, 15), (20440224.0, 2013.0): (18.0, 4), (20438809.0, 2012.0): (52.0, 25), (20441583.0, 2013.0): (1.0, 8), (20427583.0, 2013.0): (9.0, 5), (20441561.0, 2014.0): (42.0, 6), (20438953.0, 2013.0): (33.0, 24), (20440561.0, 2013.0): (52.0, 24), (20441583.0, 2012.0): (52.0, 10), (20438734.0, 2014.0): (9.0, 25), (20427504.0, 2014.0): (42.0, 20), (20442216.0, 2014.0): (6.0, 25),

In [72]:
answer([2014, 50, 20438756, 1, 2, 3, 4, 5, 6, 7, 8, 9])

20438756


-1

In [73]:
def answer(x):
    year = x[0]
    week = x[1]
    item_id = x[2]
    print item_id
    if (item_id, year) in d.keys():
        ans = d[(item_id, year)]
        ans_week = ans[0]
        ans_f = ans[1]
        f = [0 for i in range(30)]
        f[ans_f - 1] = ans_week
        cur = ans_week
        for i in range(0, ans_f - 1):
            f[ans_f - 2 - i] = ans_week + 1 + i
        for i in range(0, 30 - ans_f):
            f[ans_f + i] = ans_week - 1 - i
        for i in range(30):
            if f[i] <= 0:
                f[i] = f[i] + 53
            elif f[i] > 53:
                    f[i] = f[i] - 53
        print f
        for i in range(30):
            if f[i] == week:
                return x[i + 3]
    return -1

In [58]:
train = train.sample(frac=0.1, random_state=57)
n = train.shape[0]
F = train.drop(['Num', 'y'], axis=1)

y = train['y'].reshape(n, 1)
y = [elem[0] for elem in y]
#machine learning part
for tress_number in [30]:
    for depth in [7]:
        for random_state in [43]:
            model = RandomForestRegressor(n_estimators=tress_number, max_depth=depth,
                                              random_state=random_state, criterion='mae')
#             accuracy = cross_val_score(model, X, y, scoring=scorer, cv=tscv)
#             print(accuracy, tress_number, depth, random_state)

def smape(y, y_pred, **kwargs):
    y = np.array(y)
    y_pred = np.array(y_pred)
    ans = (np.abs(y - y_pred) * 1.0 / (np.abs(y) + np.abs(y_pred))).mean() * 100
    return ans * 2

from sklearn.metrics import make_scorer
scorer = make_scorer(score_func=smape)
tscv = TimeSeriesSplit(n_splits=2)

accuracy = cross_val_score(model, F, y, scoring=scorer, cv=tscv)
print accuracy
#model.fit(F, y)

/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


[ 31.56420165  30.95706907]


In [59]:
model.fit(F, y)

RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=43,
           verbose=0, warm_start=False)

In [74]:
N = test.shape[0]

test['week'] = test['week'] - test['shift']

for i in range(N):
    if test['week'].values[i] <= 0:
        test['week'].values[i] = (test['week'].values[i] + 52) % 53
        test['year'].values[i] = (test['year'].values[i] - 1)

test = test.drop(['shift'], axis=1)

X = test.drop(['Num'], axis=1)

In [75]:
X = np.array(X)

preds = []
for x in X:
    ans = answer(x)
    p = model.predict([x])[0]
    if ans != -1:
        preds.append(ans)
    else:
        preds.append(p)

print preds
print len(preds)
print len(sample_submission)

20447918.0
20447902.0
20447732.0
20443951.0
20443944.0
20443943.0
20443941.0
20443902.0
20443901.0
20443308.0
20443307.0
20443057.0
20443056.0
20443055.0
20447919.0
20447972.0
20447973.0
20448083.0
20448084.0
20448086.0
[1.0, 53.0, 52.0, 51.0, 50.0, 49.0, 48.0, 47.0, 46.0, 45.0, 44.0, 43.0, 42.0, 41.0, 40.0, 39.0, 38.0, 37.0, 36.0, 35.0, 34.0, 33.0, 32.0, 31.0, 30.0, 29.0, 28.0, 27.0, 26.0, 25.0]
20448095.0
20448162.0
20448312.0
[24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 53.0, 52.0, 51.0, 50.0, 49.0, 48.0]
20448455.0
20448467.0
20448805.0
20448900.0
20448958.0
20441997.0
20441382.0
20441561.0
[47.0, 46.0, 45.0, 44.0, 43.0, 42.0, 41.0, 40.0, 39.0, 38.0, 37.0, 36.0, 35.0, 34.0, 33.0, 32.0, 31.0, 30.0, 29.0, 28.0, 27.0, 26.0, 25.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18.0]
20441562.0
20441563.0
20441781.0
20441789.0
20441790.0
20441951.0
20441952.0
20441953.0
20441988.0
20441989.0
20441990.0
20443

In [64]:
sample_submission['y'] = preds
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
sample_submission.to_csv("smart_submission.tsv", sep=',', index=False)